In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from io import StringIO
import sys
import requests
from bs4 import BeautifulSoup
import pickle
import os
import glob
from pathlib import Path
import folium

In [2]:
shs_df = pickle.load(open("data/covers.p", "rb"))

In [3]:
shs_df.head()

,trackID,artistID,shsPerf,clique_id,name,title,year
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,-1,0,Louis Armstrong,Stardust,1988.0
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,-1,0,Artie Shaw and his orchestra,Stardust,1988.0
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,-1,0,Hoagy Carmichael,Star Dust,1999.0
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,-1,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,NaN
4,TROJZTF128F428B546,ARJN76O1187FB43C99,-1,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,2001.0


Create dataframe of unique artists with their location

In [4]:
artists = list(shs_df['name'])
unique_artists = pd.DataFrame(list(np.unique(artists)), columns = ["name"])

In [5]:
import unidecode
def format_string(x):
    return unidecode.unidecode(x).lower().replace("&", "and").replace("_", " ").replace("'", "").replace(".", "")

In [6]:
def get_artist_location(artist):
    artist = artist
    # Handle some special cases
    if artist == "98º":
        artist = "98 degrees"
    if artist == "4hero":
        artist = "4 hero"
    if artist == "? & The Mysterians":
        artist = "the mysterians"
    if artist == "crosby_ stills":
        artist = "crosby stills"
    r = requests.get('https://secondhandsongs.com/search?search_text='+format_string(artist))
    soup = BeautifulSoup(r.text, 'html.parser')
    if "Search" in (soup.find("head")).find("title").text:
        table = soup.find('section', id="section-artist")
        try :
            title = table.find('h2', id='t1')
        except:
            return "Unavailable"
        # Check if the result is an artist
        if "Artists" in title.text:
        # Get the artist ID to find the its web page on SHS
            # When only one artist is found
            if "1+" in title.text or title.text == "Artists 1":
                if "Mysterians" in artist:
                    print("mysterians 1+")
                artist_id = str(table.a).split("\"")[1]
            # When several artists are found
            else:
                if "Mysterians" in artist:
                    print("mysterians bug number results")
                artists = table.find_all("a")
                artist_id = ""
                for art in artists:
                    if art.text != "\n\n":
                        if format_string(artist) in format_string(art.text):
                            artist_id = str(art).split("\"")[1]
                            break
                if artist_id == "":
                    return "Unavailable"
    
            r_artist = requests.get('https://secondhandsongs.com'+artist_id)
            soup = BeautifulSoup(r_artist.text, 'html.parser')
        else:
            return "Unavailable"
    try:
        country = soup.find("dd", itemprop="nationality").text
        return country
    except:
        return "Missing"

In [7]:
#unique_artists["country"] = unique_artists["name"].map(lambda x : get_artist_location(x))
#pickle.dump(unique_artists, open("data/first_location.p", "wb"))
unique_artists = pickle.load(open("data/first_location.p", "rb"))

In [8]:
unknown_loc = unique_artists[unique_artists['country'] == "Unavailable"].reset_index()
unknown_loc.head(10)

,index,name,country
0,1,1 Giant Leap feat. Michael Stipe & Asha Bhosle,Unavailable
1,17,50 Cent ft. Young Buck,Unavailable
2,29,A*Teens,Unavailable
3,33,A3,Unavailable
4,50,Ace Karaoke Productions,Unavailable
5,63,Adema,Unavailable
6,94,Alabama 3 / Tenor Fly,Unavailable
7,113,Alexandra Burke Feat. Ne-Yo,Unavailable
8,125,Alison Krauss / Union Station,Unavailable
9,126,Alison Limerick,Unavailable


By looking at the data for which we did not find a country, we observe that for a lot of them this is due to the fact that they are not single artists but featurings. We then decide to assign them the country of the first artist in the featuring. For those cases, we establish a list of separators that represent a featuring (e.g. "feat", "and", or "/"). Then we keep only the first artist in the list, and we proceed as before to find its country.

In [9]:
def location_featuring(artist):
    sep_list = ["with", ";", " ft. ", "feat", " and ", "/", "&"]
    sep = ""
    i = 0
    country = ""
    while (sep == "") and i < len(sep_list):
        if sep_list[i] in artist.lower():
            sep = sep_list[i]
            artist = artist.lower().split(sep)[0].replace("_", "")
            if artist[-1] == " ":
                artist = artist[:-1]
            country = get_artist_location(artist)
            break
        i = i+1
    if country == "":
        return "Unavailable"
    else:
        return country
                    
    

In [10]:
'''
feat_loc = unknown_loc[['name']]
feat_loc['country'] = feat_loc['name'].map(lambda x : location_featuring(x))
for n in feat_loc2['name']:
    i = unique_artists[unique_artists['name'] == n].index[0]
    if unique_artists.get_value(i, 'country') == "Unavailable":
        unique_artists.set_value(i, 'country', feat_loc2.get_value(feat_loc2[feat_loc2['name'] == n].index[0], 'country'))

pickle.dump(unique_artists, open("data/artists_location.p", "wb"))
'''

unique_artists = pickle.load(open("data/artists_location.p", "rb"))

In [11]:
len(unique_artists[unique_artists['country'] == "Unavailable"])

201

In [15]:
#shs_df['country'] = shs_df['name'].map(lambda x : unique_artists.get_value(unique_artists[unique_artists['name'] == x].index[0], 'country'))
#pickle.dump(shs_df, open("data/covers_loc.p", "wb"))
shs_df = pickle.load(open("data/covers_loc.p", "rb"))
print("Number of tracks for which we didn't manage to webscrap the country : " + str(len(shs_df[shs_df['country'] == "Unavailable"])))
print("Number of tracks for which we country was missing on SHS : " + str(len(shs_df[shs_df['country'] == "Missing"])))

Number of tracks for which we didn't manage to webscrap the country : 457
Number of tracks for which we country was missing on SHS : 954


In [13]:
count_countries = shs_df.groupby('country')[['country']].count().sort_values('country', ascending=False)
count_countries.columns = ['count']
count_countries.drop(['Missing', 'Unavailable'], inplace=True)
count_countries

,count
country,
United States,11191
United Kingdom,3177
Canada,482
France,232
Germany,201
Australia,185
Sweden,182
Italy,179
Ireland,166


In [16]:
unique_artists[unique_artists['country'] == "Missing"]

,name,country
3,10cc,Missing
8,2Pac,Missing
14,4 Non Blondes,Missing
51,Ace Moreland,Missing
53,Across The Universe,Missing
54,Across the Border,Missing
64,Adeva,Missing
98,Alastair Galbraith,Missing
107,Alessandro Safina,Missing
148,Ambidextrous,Missing


In [17]:
artists_location = pickle.load(open("data/data_country.p", "rb"))
artists_location

,shsID,trackID,artistID,shsPerf,artist,location,country
1,"115402,74782, Putty (In Your Hands)",TRNJXCO128F92E1930,ARQD13K1187B98E441,138584,Sylvie Vartan,"Iskretz, Bulgaria",Bulgaria
2,"24350, I.G.Y. (Album Version)",TRIBOIS128F9340B19,ARUVZYG1187B9B2809,24350,Donald Fagen,"Passaic, NJ",United States of America
7,"11012, Sheer Heart Attack",TRABVTG128F934AB80,AR9BVRM1187FB51139,97131,Hallows Eve,Georgia,Georgia
8,"11012, Sheer Heart Attack",TRRZZZZ128F422F784,ARNFBNR1187B9A25C2,-1,Helloween,"Hamburg, Germany",Germany
11,"10974, Standing At The Crossroads",TRNTRUC128F4234EB5,AR49MOS1187B991A8B,139493,Smokey Wilson,"Glen Allen, MS",United States of America
16,"10974, Standing At The Crossroads",TROKQBW128F92FACDF,AR5U94U1187FB5CDCF,16206,John Hammond,"Nashville, TN",United States of America
17,"10974, Standing At The Crossroads",TRKGFVN128F4250F29,ARH1XAK1187B98F188,38920,Robin Trower,"London, England",United Kingdom
18,"10974, Standing At The Crossroads",TRVJQRV128F4214321,ARZHSES1187FB58AAC,10975,Cream,England,United Kingdom
19,"10974, Standing At The Crossroads",TRTGOUT128F92F9590,ARQ8BBK1187FB49C9A,90772,"James ""Son"" Thomas & Cleveland ""Broomman"" Jones","Eden, MS",United States of America
21,"42790, The Hustle Is On",TRGYGGS12903CF2C06,AR8Q4RZ1187FB46BDC,42790,Dinah Washington,"Linden, TX",United States of America


In [18]:
unique_artists['id'] = unique_artists['name'].map\
(lambda x : shs_df.get_value(shs_df[shs_df['name'] == x].head(1).index[0], 'artistID'))

In [50]:
def complete_location(value):
    for a_id in unique_artists[unique_artists['country'] == value]['id']:
        if not artists_location[artists_location['artistID'] == a_id].empty:
            index_unique_art = unique_artists[unique_artists['id'] == a_id].index[0]
            index_art_loc = artists_location[artists_location['artistID'] == a_id].index[0]
            country = artists_location.get_value(index_art_loc, 'country')
            unique_artists.set_value(index_unique_art, 'country', country)
            print(str(unique_artists.get_value(index_unique_art, 'name')) + " => " + str(country))
            

complete_location("Missing")
complete_location("Unavailable")

In [51]:
print("Number of artists for which we didn't manage to webscrap the country : " + str(len(unique_artists[unique_artists['country'] == "Unavailable"])))
print("Number of artists for which we country was missing on SHS : " + str(len(unique_artists[unique_artists['country'] == "Missing"])))

Number of artists for which we didn't manage to webscrap the country : 159
Number of artists for which we country was missing on SHS : 404


In [59]:
#shs_df['country'] = shs_df['name'].map(lambda x : unique_artists.get_value(unique_artists[unique_artists['name'] == x].index[0], 'country'))
#pickle.dump(shs_df, open("data/covers_loc.p", "wb"))
shs_df = pickle.load(open("data/covers_loc.p", "rb"))

In [60]:
len(shs_df[shs_df['country']== "Missing"])

684

In [61]:
len(shs_df[shs_df['country']== "Unavailable"])

310

In [62]:
import folium

In [79]:
test = folium.Map([54,15], zoom_start = 1.5)
json = json.load(open("data/custom.geo.json"))

In [83]:
folium.GeoJson(json).add_to(test)
test

In [84]:
count_countries.reset_index(inplace=True)
count_countries

,country,count
0,United States,11191
1,United Kingdom,3177
2,Canada,482
3,France,232
4,Germany,201
5,Australia,185
6,Sweden,182
7,Italy,179
8,Ireland,166
9,Jamaica,117


In [98]:
song_repartition_map = folium.Map()
song_repartition_map.choropleth(
    geo_data = json,
    data=count_countries,
    columns=['country', 'count'],
    key_on='properties.name',
    fill_color = 'OrRd', fill_opacity=0.7,line_opacity=0.9,
    line_color='white',
    legend_name="Unemployment in Europe (in % of population)",
    threshold_scale = [0,1,100,200,5000,10000]
    )

In [99]:
song_repartition_map